# Page 1-3

In [1]:
import time
import random
import re
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ========== 1. User-Agent 列表（仅桌面端 Chrome） ==========
# 🚀 伪装 User-Agent
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_6_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Fedora) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_7_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; ARM64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_6_8) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Arch) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_7_10) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; ZorinOS) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Elementary OS) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; MX Linux) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko Chrome/120.0.6350.90 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6350.90 Safari/537.36"
]




random_user_agent = random.choice(USER_AGENTS)

# ========== 2. 初始化 WebDriver ==========
service = Service("/Users/zhuxiaowen/Desktop/chromedriver-mac-arm64/chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument(f"user-agent={random_user_agent}")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=service, options=options)
print(f"✅ 启动 WebDriver, 使用UA: {random_user_agent}")

# ========== 3. 访问页面 ==========
base_url = "https://www.target.com/s?searchTerm=face+mask&tref=typeahead%7Cterm%7Cface+mask%7C%7C%7Chistory&moveTo=product-list-grid"
driver.get(base_url)
time.sleep(random.uniform(5, 10))

# ========== 4. 修改点①：等待商品卡片加载完成 ==========
def wait_until_products_loaded(min_count=25, timeout=15):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: len(d.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")) >= min_count
        )
    except TimeoutException:
        print(f"⚠️ 超时未加载出 {min_count} 个商品，当前页可能不完整")

# ========== 5. 修改点②：强化滚动逻辑，直到商品数量不再增加 ==========
def scroll_and_wait():
    last_height = driver.execute_script("return document.body.scrollHeight")
    prev_count = 0

    for i in range(30):
        print(f"🔄 滚动页面: 第 {i+1} 次")
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(random.uniform(2, 4))

        products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
        if len(products) > prev_count:
            prev_count = len(products)
        else:
            print("✅ 页面懒加载完成")
            break

        try:
            if products:
                ActionChains(driver).move_to_element(products[-1]).perform()
        except:
            pass

    time.sleep(3)

# ========== 🔧 修改点①：记录每一页的首个商品链接 ==========
visited_first_links = set()

product_list = []

def scrape_products():
    global product_list, visited_first_links
    scroll_and_wait()
    wait_until_products_loaded()

    products = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")
    print(f"🔍 当前页面商品数: {len(products)}")

    if not products:
        print("⚠️ 本页无商品，跳过")
        return

    # 🔧 修改点②：判断是否回到了已访问页面（用首个商品链接判重）
    try:
        first_link = products[0].find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
    except:
        first_link = None

    if first_link in visited_first_links:
        print("🔁 检测到页面内容重复，疑似回到了第一页，停止抓取")
        raise StopIteration
    visited_first_links.add(first_link)

    for product in products:
        try:
            title_element = product.find_element(By.XPATH, ".//a[@data-test='product-title']//div")
            name = title_element.text.strip()
        except NoSuchElementException:
            continue

        try:
            brand = product.find_element(By.XPATH, ".//a[@data-test='@web/ProductCard/ProductCardBrandAndRibbonMessage/brand']").text.strip()
        except:
            brand = "N/A"

        try:
            price = product.find_element(By.XPATH, ".//span[@data-test='current-price']").text.strip()
        except:
            price = "N/A"

        try:
            unit_price = product.find_element(By.XPATH, ".//span[@data-test='unit-price']").text.strip()
        except:
            unit_price = "N/A"

        try:
            rating = product.find_element(By.XPATH, ".//span[@data-test='ratings']").text.strip()
        except:
            rating = "N/A"

        try:
            reviews = product.find_element(By.XPATH, ".//span[@data-test='rating-count']").text.strip()
        except:
            reviews = "0 reviews"

        try:
            product.find_element(By.XPATH, ".//div[@data-test='comparison-price']")
            is_on_sale = 1
        except:
            is_on_sale = 0

        try:
            if " - " in price:
                is_free_shipping = "N/A"
            else:
                price_value = float(price.replace("$", ""))
                is_free_shipping = 1 if price_value >= 35 else 0
        except:
            is_free_shipping = "N/A"

        try:
            shipping_info = product.find_element(By.XPATH, ".//div[@data-test='LPFulfillmentSectionShippingFA_standardShippingMessage']").text
            if "get it tomorrow" in shipping_info:
                is_3day_delivery = 1
            else:
                match = re.search(r"arrives\s+\w+,\s+([A-Za-z]+ \d+)", shipping_info)
                if match:
                    delivery_str = match.group(1)
                    delivery_date = datetime.datetime.strptime(f"{delivery_str} {datetime.datetime.now().year}", "%b %d %Y").date()
                    days_diff = (delivery_date - datetime.datetime.now().date()).days
                    is_3day_delivery = 1 if 0 <= days_diff <= 3 else 0
                else:
                    is_3day_delivery = "N/A"
        except:
            is_3day_delivery = "N/A"

        try:
            link = product.find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
        except:
            link = "N/A"

        product_list.append({
            "product_name": name,
            "brand": brand,
            "current_price": price,
            "unit_price": unit_price,
            "star_rating": rating,
            "review_counts": reviews,
            "is_on_sale": is_on_sale,
            "is_free_shipping": is_free_shipping,
            "is_3day_delivery": is_3day_delivery,
            "product_category": "Face Mask",
            "product_link": link
        })

# 翻页逻辑（原样）
def go_to_next_page(prev_first_link):
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test='next']"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        print("✅ 点击下一页")
    except:
        print("❌ 没有找到下一页按钮，结束爬取")
        return False

    # 等待新页面加载且首个商品链接发生变化
    for _ in range(15):
        time.sleep(random.uniform(1.5, 2.5))
        try:
            products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
            if products:
                new_first_link = products[0].get_attribute("href")
                if new_first_link != prev_first_link:
                    print("🔁 成功加载到新页面")
                    return True
        except:
            pass
    print("⚠️ 页面跳转失败或未加载新内容")
    return False


# ========== 主循环（加异常捕捉 StopIteration） ==========
max_pages = 3
try:
    for page_num in range(max_pages):
        print(f"\n📄 正在爬取第 {page_num + 1} 页...")
        scrape_products()

        # 获取当前页面首个链接作为翻页后的参照
        current_first_link = next(iter(visited_first_links), None)
        if not go_to_next_page(current_first_link):
            break
except StopIteration:
    print("⛔ 检测到重复页，主动中止爬虫")


driver.quit()
print("✅ WebDriver 关闭")

print(f"📦 共抓取商品（去重后）：{len(product_list)} 条")

✅ 启动 WebDriver, 使用UA: Mozilla/5.0 (X11; Linux x86_64; Arch) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36

📄 正在爬取第 1 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 2 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 3 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面
✅ WebDriver 关闭
📦 共抓取商品（去重后）：74 条


In [16]:
df = pd.DataFrame(product_list)
df = df[df['product_name'].str.contains('oz', case=False, na=False)]
df = df.reset_index(drop=True)
df

product_name                 brand  \
0   Que Bella Exfoliating Strawberry Peel Off Mask...             Que Bella   
1             Avatara Drink Up Face Mask - 0.71 fl oz               Avatara   
2            Avatara Chill Out Face Mask - 0.71 fl oz               Avatara   
3   Avatara Vitamin C Water Gel Face Mask - 0.71 f...               Avatara   
4   Pacifica Vegan Collagen Hydrate and Plump Faci...              Pacifica   
5   Neutrogena Clear Pore 2 in 1 Facial Cleanser/F...            Neutrogena   
6   Pacifica Stress Rehab Coconut and Caffeine Fac...              Pacifica   
7   Que Bella 2% Collagen Solution Cream Face Mask...             Que Bella   
8   Avatara Niacinamide Water Gel Face Mask - 0.71...               Avatara   
9   TULA SKINCARE Claycation Detoxing & Toning Fac...         TULA SKINCARE   
10      Avatara Rose Boba Hydrogel Face Mask - 0.98oz               Avatara   
11               LOOPS Dream Sleep Face Mask - 1.27oz                 LOOPS   
12  ESW Beauty Deep Detox Pore Control Raw Juice F...            ESW Beauty   
13  LOOPS Hyper Smooth Dark Spot Treatment Hydroge...                 LOOPS   
14  Pacifica Super Green Detox Kale and Charcoal F...              Pacifica   
15  Kiehl's Ultra Facial Overnight Hydrating Face ...               Kiehl's   
16  Origins Clear Improvement Blackhead Clearing C...               Origins   
17  Avatara Lychee Boba Hydrogel Face Mask - 0.63 ...               Avatara   
18  Rael Miracle Clear Pore Purifying Kaolin Clay ...                  Rael   
19       Proactiv Skin Purifying Acne Face Mask - 3oz              Proactiv   
20  Seoul Ceuticals Korean Skin Care Green Tea Fac...       Seoul Ceuticals   
21  Seoul Ceuticals Korean Skin Care Korean Face M...       Seoul Ceuticals   
22  Que Bella 2% Collagen Solution Cream Face Mask...             Que Bella   
23       Proactiv Skin Purifying Acne Face Mask - 3oz              Proactiv   
24  Seoul Ceuticals Korean Skin Care Green Tea Fac...       Seoul Ceuticals   
25  Seoul Ceuticals Korean Skin Care Korean Face M...       Seoul Ceuticals   
26  Origins Drink Up Intensive Overnight Hydrating...               Origins   
27     Que Bella Cleansing Pink Clay Mud Mask - 0.5oz             Que Bella   
28  Origins Drink Up Intensive Overnight Hydrating...               Origins   
29  Pacifica Future Youth Gravity Rebound Face Mas...              Pacifica   
30  Seoul Ceuticals Korean Skin Care Exfoliating C...       Seoul Ceuticals   
31  Admire My Skin Vitamin C Mask For Face - Brigh...        Admire My Skin   
32  Seoul Ceuticals Korean Skin Care Hyaluronic Ac...       Seoul Ceuticals   
33  Andalou Naturals Age Defying Bio-Active 8 Berr...      Andalou Naturals   
34  Que Bella Detoxifying Charcoal Peel off Mask -...             Que Bella   
35  Oh K! Slide & Seal Copper Peptides Eye Mask wi...                 Oh K!   
36  Neutrogena Hydro Boost Moisturizing Sheet Mask...            Neutrogena   
37  Curology Gel Moisturizer, Lightweight Daily Fa...              Curology   
38  Versed Doctors Visit Instant Resurfacing Mask ...                Versed   
39    Avatara Watermellow Hydrating Mask - 0.71 fl oz               Avatara   
40  Pacifica Glow Baby Brightening Facial Mask - 0...              Pacifica   
41  Face Mask With White and Green Clay by LErbola...            LErbolario   
42        Blesswell Blue Charcoal Face Mask - 2 fl oz             Blesswell   
43  Que Bella Exfoliating Strawberry Peel Off Mask...             Que Bella   
44  Coming Up Roses Exfoliating Face Mask, Exfolia...  The Organic Skin Co.   
45        LOOPS Variety Pack Facial Mask - 5ct/5.29oz                 LOOPS   
46          Que Bella Soothing Honey Gel Mask - 0.5oz             Que Bella   
47       Que Bella Relaxing Lavender Mud Mask - 0.5oz             Que Bella   

   current_price unit_price                           star_rating  \
0          $2.29        N/A  4.5 out of 5 stars with 1247 ratings   
1          $3.59        N/A   4

In [8]:
df.to_csv("facemask_data.csv",index=False)

# Page 4-6

In [5]:
import time
import random
import re
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ========== 1. User-Agent 列表（仅桌面端 Chrome） ==========
# 🚀 伪装 User-Agent
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_6_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Fedora) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_7_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; ARM64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_6_8) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Arch) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_7_10) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; ZorinOS) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Elementary OS) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; MX Linux) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko Chrome/120.0.6350.90 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6350.90 Safari/537.36"
]




random_user_agent = random.choice(USER_AGENTS)

# ========== 2. 初始化 WebDriver ==========
service = Service("/Users/zhuxiaowen/Desktop/chromedriver-mac-arm64/chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument(f"user-agent={random_user_agent}")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=service, options=options)
print(f"✅ 启动 WebDriver, 使用UA: {random_user_agent}")

# ========== 3. 访问页面 ==========
base_url = "https://www.target.com/s?searchTerm=face+mask&tref=typeahead%7Cterm%7Cface+mask%7C%7C%7Chistory&moveTo=product-list-grid&Nao=72"
driver.get(base_url)
time.sleep(random.uniform(5, 10))

# ========== 4. 修改点①：等待商品卡片加载完成 ==========
def wait_until_products_loaded(min_count=25, timeout=15):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: len(d.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")) >= min_count
        )
    except TimeoutException:
        print(f"⚠️ 超时未加载出 {min_count} 个商品，当前页可能不完整")

# ========== 5. 修改点②：强化滚动逻辑，直到商品数量不再增加 ==========
def scroll_and_wait():
    last_height = driver.execute_script("return document.body.scrollHeight")
    prev_count = 0

    for i in range(30):
        print(f"🔄 滚动页面: 第 {i+1} 次")
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(random.uniform(2, 4))

        products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
        if len(products) > prev_count:
            prev_count = len(products)
        else:
            print("✅ 页面懒加载完成")
            break

        try:
            if products:
                ActionChains(driver).move_to_element(products[-1]).perform()
        except:
            pass

    time.sleep(3)

# ========== 🔧 修改点①：记录每一页的首个商品链接 ==========
visited_first_links = set()

product_list2 = []

def scrape_products():
    global product_list2, visited_first_links
    scroll_and_wait()
    wait_until_products_loaded()

    products = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")
    print(f"🔍 当前页面商品数: {len(products)}")

    if not products:
        print("⚠️ 本页无商品，跳过")
        return

    # 🔧 修改点②：判断是否回到了已访问页面（用首个商品链接判重）
    try:
        first_link = products[0].find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
    except:
        first_link = None

    if first_link in visited_first_links:
        print("🔁 检测到页面内容重复，疑似回到了第一页，停止抓取")
        raise StopIteration
    visited_first_links.add(first_link)

    for product in products:
        try:
            title_element = product.find_element(By.XPATH, ".//a[@data-test='product-title']//div")
            name = title_element.text.strip()
        except NoSuchElementException:
            continue

        try:
            brand = product.find_element(By.XPATH, ".//a[@data-test='@web/ProductCard/ProductCardBrandAndRibbonMessage/brand']").text.strip()
        except:
            brand = "N/A"

        try:
            price = product.find_element(By.XPATH, ".//span[@data-test='current-price']").text.strip()
        except:
            price = "N/A"

        try:
            unit_price = product.find_element(By.XPATH, ".//span[@data-test='unit-price']").text.strip()
        except:
            unit_price = "N/A"

        try:
            rating = product.find_element(By.XPATH, ".//span[@data-test='ratings']").text.strip()
        except:
            rating = "N/A"

        try:
            reviews = product.find_element(By.XPATH, ".//span[@data-test='rating-count']").text.strip()
        except:
            reviews = "0 reviews"

        try:
            product.find_element(By.XPATH, ".//div[@data-test='comparison-price']")
            is_on_sale = 1
        except:
            is_on_sale = 0

        try:
            if " - " in price:
                is_free_shipping = "N/A"
            else:
                price_value = float(price.replace("$", ""))
                is_free_shipping = 1 if price_value >= 35 else 0
        except:
            is_free_shipping = "N/A"

        try:
            shipping_info = product.find_element(By.XPATH, ".//div[@data-test='LPFulfillmentSectionShippingFA_standardShippingMessage']").text
            if "get it tomorrow" in shipping_info:
                is_3day_delivery = 1
            else:
                match = re.search(r"arrives\s+\w+,\s+([A-Za-z]+ \d+)", shipping_info)
                if match:
                    delivery_str = match.group(1)
                    delivery_date = datetime.datetime.strptime(f"{delivery_str} {datetime.datetime.now().year}", "%b %d %Y").date()
                    days_diff = (delivery_date - datetime.datetime.now().date()).days
                    is_3day_delivery = 1 if 0 <= days_diff <= 3 else 0
                else:
                    is_3day_delivery = "N/A"
        except:
            is_3day_delivery = "N/A"

        try:
            link = product.find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
        except:
            link = "N/A"

        product_list2.append({
            "product_name": name,
            "brand": brand,
            "current_price": price,
            "unit_price": unit_price,
            "star_rating": rating,
            "review_counts": reviews,
            "is_on_sale": is_on_sale,
            "is_free_shipping": is_free_shipping,
            "is_3day_delivery": is_3day_delivery,
            "product_category": "Face Mask",
            "product_link": link
        })

# 翻页逻辑（原样）
def go_to_next_page(prev_first_link):
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test='next']"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        print("✅ 点击下一页")
    except:
        print("❌ 没有找到下一页按钮，结束爬取")
        return False

    # 等待新页面加载且首个商品链接发生变化
    for _ in range(15):
        time.sleep(random.uniform(1.5, 2.5))
        try:
            products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
            if products:
                new_first_link = products[0].get_attribute("href")
                if new_first_link != prev_first_link:
                    print("🔁 成功加载到新页面")
                    return True
        except:
            pass
    print("⚠️ 页面跳转失败或未加载新内容")
    return False


# ========== 主循环（加异常捕捉 StopIteration） ==========
max_pages = 3
try:
    for page_num in range(max_pages):
        print(f"\n📄 正在爬取第 {page_num + 1} 页...")
        scrape_products()

        # 获取当前页面首个链接作为翻页后的参照
        current_first_link = next(iter(visited_first_links), None)
        if not go_to_next_page(current_first_link):
            break
except StopIteration:
    print("⛔ 检测到重复页，主动中止爬虫")


driver.quit()
print("✅ WebDriver 关闭")

print(f"📦 共抓取商品（去重后）：{len(product_list2)} 条")

✅ 启动 WebDriver, 使用UA: Mozilla/5.0 (Windows NT 10.0; ARM64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36

📄 正在爬取第 1 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 2 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 3 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面
✅ WebDriver 关闭
📦 共抓取商品（去重后）：87 条


In [17]:
df2 = pd.DataFrame(product_list2)
df2 = df2[df2['product_name'].str.contains('oz', case=False, na=False)]
df2 = df2.reset_index(drop=True)
df2

product_name            brand  \
0   e.l.f. Holy Hydration! Broad Spectrum Sunscree...           e.l.f.   
1   Que Bella Detoxifying Charcoal Peel off Mask -...        Que Bella   
2   Oh K! Strawberry & Mint Jelly Foot Peel with S...            Oh K!   
3     e.l.f. Daily Hydration Moisturizer - 2.53 fl oz           e.l.f.   
4        BYOMA Reviving Retinol Face Oil - 1.01 fl oz            BYOMA   
5   Spa Sciences Hydrating Detox Mask Purifying Tr...     Spa Sciences   
6   Azure Skincare Collagen and Peptides Sheet Mas...   Azure Skincare   
7   Oh K! Triple Hyaluronic Slugging Mask with Aze...            Oh K!   
8   ESW Beauty Deep Detox Pore Control Raw Juice M...       ESW Beauty   
9   L'Erbolario Hyaluronic Acid Triple Action Face...       LErbolario   
10           LOOPS Glow Gang Facial Mask - 3ct/1.18oz            LOOPS   
11                          Sun Bum Facial Mask - 2oz          Sun Bum   
12  Versed Doctors Visit Instant Resurfacing Mask ...           Versed   
13         LOOPS Clean Slate Facial Mask - 5ct/5.29oz            LOOPS   
14  Tyro Supreme Anti-Age Mask - Face Mask for Bea...             Tyro   
15  Pacifica Stress Rehab Coconut and Caffeine Fac...         Pacifica   
16  Que Bella Hydrating Pineapple Peel Off Mask - ...        Que Bella   
17  Que Bella Refreshing Watermelon Peel Off Mask ...        Que Bella   
18         Que Bella Pore Minimizing Mud Mask - 0.5oz        Que Bella   
19  Origins Original Skin Retexturizing Mask - 2.5...          Origins   
20          Que Bella Soothing Honey Gel Mask - 0.5oz        Que Bella   
21  Oh K! Vitamin C Sheet Mask with Active Powder ...            Oh K!   
22  Tyro Supreme Anti-Age Mask - Face Mask for Bea...             Tyro   
23                LOOPS Skin Cycle Trio Mask - 1.75oz            LOOPS   
24      La Prairie Skin Caviar Luxe Sleep Mask 1.7 oz       La Prairie   
25  St. Ives Apricot Face Exfoliant for Balanced t...         St. Ives   
26     Que Bella 1% Vitamin C Sheet Mask - 0.63 fl oz        Que Bella   
27     LOOPS Weekly Reset Rejuvenating Mask - 1.058oz            LOOPS   
28       Que Bella Cooling Avocado Clay Mask  - 0.5oz        Que Bella   
29  Oh K! Vitamin C Sheet Mask with Active Powder ...            Oh K!   
30  Tyro Mineral Relaxing Mask - Face Mask for Bea...             Tyro   
31          Oh K! Heartleaf Semi-Gel Mask - 0.7 fl oz            Oh K!   
32  I DEW CARE Space Kitten Exfoliating Galactic B...       I DEW CARE   
33         TONYMOLY Lavender Face Sheet Mask - 0.74oz         TONYMOLY   
34                  LOOPS Under Eye Mask - 5ct/0.88oz            LOOPS   
35  Tyro Night Mask E - Face Mask for Beauty - 1.6...             Tyro   
36  Urban Hydration Bright & Balanced Aloe Gel Fac...  Urban Hydration   
37  Dr. Jart+ Vital Hydra Solution Sheet-type Faci...        Dr. Earth   
38                     Pursonic Honey Facial Mask 8oz         Pursonic   

   current_price           unit_price                           star_rating  \
0         $13.00        ($7.65/ounce)  4.4 out of 5 stars with 1501 ratings   
1          $2.99                  N/A  4.3 out of 5 stars with 1116 ratings   
2          $6.79                  N/A    4.6 out of 5 stars with 13 ratings   
3          $8.00  ($3.20/fluid ounce)  4.3 out of 5 stars with 1368 ratings   
4         $21.99                  N/A    4.4 out of 5 stars with 54 ratings   
5         $11.99  ($6.00/fluid ounce)     4.8 out of 5 stars with 6 ratings   
6          $7.99                  N/A    4.3 out of 5 stars with 12 ratings   
7          $5.99                  N/A      5 out of 5 stars with 12 ratings   
8         $26.99                  N/A                                   N/A   
9         $27.79                  N/A                                   N/A   
10        $21.99                  N/A    4.8 out of 5 stars with 80 ratings   
11         $9.99                  N/A   4.4 out of 5 stars with 106 ratings   
12        $21.99                  N/A   4.3 out of 5 

In [18]:
df2.to_csv("facemask_data2.csv",index=False)

In [19]:
df_combined_facemask_1_6 = pd.concat([df,df2],axis=0, ignore_index=True)
df_combined_facemask_1_6

product_name            brand  \
0   Que Bella Exfoliating Strawberry Peel Off Mask...        Que Bella   
1             Avatara Drink Up Face Mask - 0.71 fl oz          Avatara   
2            Avatara Chill Out Face Mask - 0.71 fl oz          Avatara   
3   Avatara Vitamin C Water Gel Face Mask - 0.71 f...          Avatara   
4   Pacifica Vegan Collagen Hydrate and Plump Faci...         Pacifica   
..                                                ...              ...   
82                  LOOPS Under Eye Mask - 5ct/0.88oz            LOOPS   
83  Tyro Night Mask E - Face Mask for Beauty - 1.6...             Tyro   
84  Urban Hydration Bright & Balanced Aloe Gel Fac...  Urban Hydration   
85  Dr. Jart+ Vital Hydra Solution Sheet-type Faci...        Dr. Earth   
86                     Pursonic Honey Facial Mask 8oz         Pursonic   

   current_price unit_price                           star_rating  \
0          $2.29        N/A  4.5 out of 5 stars with 1247 ratings   
1          $3.59        N/A   4.5 out of 5 stars with 373 ratings   
2          $3.59        N/A   4.4 out of 5 stars with 387 ratings   
3          $3.99        N/A   4.5 out of 5 stars with 209 ratings   
4          $4.99        N/A   4.6 out of 5 stars with 110 ratings   
..           ...        ...                                   ...   
82        $31.49        N/A   4.9 out of 5 stars with 530 ratings   
83        $18.70        N/A                                   N/A   
84         $8.79        N/A   4.2 out of 5 stars with 163 ratings   
85        $24.99        N/A                                   N/A   
86        $16.99        N/A                                   N/A   

   review_counts  is_on_sale is_free_shipping  is_3day_delivery  \
0   1247 reviews           0                0                 1   
1    373 reviews           0                0                 1   
2    387 reviews           0                0                 1   
3    209 reviews           0                0                 1   
4    110 reviews           0                0                 1   
..           ...         ...              ...               ...   
82   530 reviews           0                0                 1   
83     0 reviews           0                0                 0   
84   163 reviews           0                0                 1   
85     0 reviews           1                0                 0   
86     0 reviews           1                0                 0   

   product_category                                       product_link  
0         Face Mask  https://www.target.com/p/que-bella-exfoliating...  
1         Face Mask  https://www.target.com/p/avatara-drink-up-face...  
2         Face Mask  https://www.target.com/p/avatara-chill-out-fac...  
3         Face Mask  https://www.target.com/p/avatara-vitamin-c-wat...  
4         Face Mask  https://www.target.com/p/pacifica-vegan-collag...  
..              ...                                                ...  
82        Face Mask  https://www.target.com/p/loops-under-eye-mask-...  
83        Face Mask  https://www.target.com/p/tyro-night-mask-e-fac...  
84        Face Mask  https://www.target.com/p/urban-hydration-brigh...  
85        Face Mask  https://www.target.com/p/dr-jart-vital-hydra-s...  
86        Face Mask  https://www.target.com/p/pursonic-honey-facial...  

[87 rows x 11 columns]

In [20]:
df_combined_facemask_1_6.to_csv("facemask_1_6.csv",index=False)